# ommx 1.9.0 リリースノート

ommx 1.9.0へようこそ！

このリリースでは、`ommx.v1.Instance` からQUBOへの変換機能が大幅に強化され、**不等式制約**と**整数変数**のサポートが追加されました。また、QUBO変換プロセスを簡単にするための新しいDriver API `to_qubo` が導入されました。

## ✨ 新機能

### 1. 不等式制約と整数変数のQUBO変換サポート

`ommx.v1.Instance` で表現された数理最適化問題からQUBO (Quadratic Unconstrained Binary Optimization) 形式への変換において、これまでサポートされていなかった不等式制約と整数変数が扱えるようになりました。

#### 整数変数のサポート ([#363](https://github.com/Jij-Inc/ommx/pull/363), [#260](https://github.com/Jij-Inc/ommx/pull/260))

整数変数を含む問題は、Log Encoding と呼ばれる手法を用いてバイナリ変数のみの問題に変換されます。これにより、整数変数を直接扱えないQUBOソルバーでも最適化計算が可能になります。

#### 不等式制約のサポート

不等式制約 ($ \sum a_i x_i \le b $ または $ \sum a_i x_i \ge b $) を含む問題をQUBOに変換するために、以下の二つの方法が実装されました。

##### 方法1 (デフォルト): 整数係数化と整数スラック変数による等式制約化 ([#366](https://github.com/Jij-Inc/ommx/pull/366))

この方法では、まず不等式制約の係数を整数に変換します。その後、整数のスラック変数を導入することで、不等式制約を等式制約 ($ \sum a'_i x_i + s = b' $) に変換します。変換された等式制約は、既存の手法を用いてペナルティ項としてQUBOの目的関数に追加されます。

**この方法がデフォルトで適用されます。**

##### 方法2: 整数スラック変数とPenalty Method ([#369](https://github.com/Jij-Inc/ommx/pull/369), [#368](https://github.com/Jij-Inc/ommx/pull/368))

この方法では、不等式制約 ($ \sum a_i x_i \le b $) に整数のスラック変数を追加します ($ \sum a_i x_i + s = b $)。ただし、方法1とは異なり、制約を等式制約として扱わずに、元の不等式制約 ($ \sum a_i x_i \le b $) を満たすようにペナルティを設計します ([Penalty Method](https://github.com/Jij-Inc/ommx/pull/368))。このペナルティ項がQUBOの目的関数に追加されます。

この方法は、後述する `to_qubo` APIの引数で `inequality_conversion_method='penalty'` を指定することで利用できます。

### 2. QUBO変換 Driver API `to_qubo` の追加 ([#370](https://github.com/Jij-Inc/ommx/pull/370))

`ommx.v1.Instance` からQUBOへの変換に必要な一連の操作（整数変数変換、不等式制約変換、ペナルティ項追加など）をまとめて実行する Driver API `to_qubo` が追加されました。これにより、ユーザーは複雑な変換ステップを意識することなく、簡単にQUBOを得ることができます。

In [ ]:
# 使用例
import ommx.v1 as ommx
from ommx.v1.driver import to_qubo

# 何らかの ommx.v1.Instance オブジェクトがあると仮定
# (例: 線形制約、二次目的関数、整数変数、不等式制約を含む)
instance: ommx.Instance = ... # ここで Instance を作成またはロード

# Driver API を使って QUBO に変換 (デフォルト設定)
qubo, offset, interpret = to_qubo(instance)

# qubo: QUBOを表す辞書 ({(i, j): coeff})
# offset: 定数オフセット
# interpret: QUBOの解を元の問題の解に解釈するための関数

print("QUBO:", qubo)
print("Offset:", offset)

# 不等式制約の変換方法として Penalty Method を指定する場合
# qubo_penalty, offset_penalty, interpret_penalty = to_qubo(instance, inequality_conversion_method='penalty')

# 必要に応じて、変換後のQUBOをソルバーに渡して最適化を実行
# result = your_qubo_solver(qubo)
# solution = interpret(result)

`to_qubo` 関数は、内部で以下のステップを適切な順序で実行します:
1. 整数変数を含む制約や目的関数をバイナリ変数表現に変換 (Log Encodingなど)
2. 不等式制約を等式制約に変換 (デフォルト) または Penalty Method 用の形式に変換
3. 等式制約や目的関数をQUBO形式に変換
4. QUBOの解を元の問題の変数にマッピングするための `interpret` 関数を生成

## 🛠️ その他の変更・改善

* (必要に応じて追記)

## 🐛 バグ修正

* (必要に応じて追記)

--- 

これらの新機能により、ommxはより広範な最適化問題をQUBO形式に変換し、様々なQUBOソルバーで解くための強力なツールとなります。ぜひ `ommx` 1.9.0 をお試しください！

フィードバックやバグ報告は、[GitHub Issues](https://github.com/Jij-Inc/ommx/issues) までお寄せください。